In [1]:
# import necessary libraries

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import train_test_split

import time

print(tf.__version__)

2.3.0


In [2]:
# define how many gpus are available and set a memmory limit
gpus = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ", len(gpus))
for i in range(len(gpus)):
    tf.config.experimental.set_virtual_device_configuration(gpus[i], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7900)]) 

Number of GPUs Available:  1


In [3]:
strategy = tf.distribute.MirroredStrategy()
# the number of replicas that is created by the strategy should be equal to the number of GPU's available
print ('Number of synchronized replicas created: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of synchronized replicas created: 1


In [4]:
# read in train and test data in case Google DRIVE is used
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# Basepath for Google DRIVE:
Basepath = '/content/drive/My Drive/Stage_ENT_Studios_2/Data/Kaggle/Arrays_5GB_float32/'

# Basepath for Jupyter notebooks:
# Basepath = 'C:/Users/lunam/Documents/1steMaster/Stage/Data_FinalArrays/Kaggle/Array_10GB/'
# also a version with 5GB and 20GB

# Basepath for KILI
# Basepath = '/home/kili/Desktop/Data_FinalArrays/Kaggle/Arrays/'

# train data
train_images = np.load(Basepath + 'train_images_Final.npy')
print('Shape train images: {}'.format(train_images.shape))

train_labels =  np.load(Basepath + 'train_labels_Final.npy')
print('Shape train labels: {}'.format(train_labels.shape))

Shape train images: (702, 512, 512, 3)
Shape train labels: (702, 2)


In [6]:
# direction where the results of the GridSearch will be stored

# direction for Google DRIVE
dir_gridsearch = '/content/drive/My Drive/Stage_ENT_Studios_2/DR_Grading/ResNet18/GridSearchResults/GridSearch_ResNet18.txt'

# direction for jupyter notebooks
# dir_gridsearch = 'C:/Users/lunam/Documents/1steMaster/Stage/Code_Final/DR_classification/DeepLearningClassification/ResNet18_GPU/GridSearchResults/Gridsearch_ResNet18.txt'

In [7]:
def Identity_Block(Input_Image, n_filters, dropout_prob = 0):


    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()
    
    # Shortcut path: Identity function
    Shortcut_Image = Input_Image
    
    # Main path, Main_Image represents the image that is passed through different layers
    Main_Image = Input_Image

    Main_Image = tf.keras.layers.Conv2D(n_filters, (3,3), kernel_initializer= W_init, padding = 'same')(Main_Image) # padding is needed to keep the same dimensions
    Main_Image = tf.keras.layers.BatchNormalization(axis = 1)(Main_Image)
    Main_Image = tf.nn.relu(Main_Image)

    Main_Image = tf.keras.layers.Conv2D(n_filters, (3,3), kernel_initializer= W_init, padding = 'same')(Main_Image)
    
    # Output: Relu activation from (Shortcut path + Main path)
    Output_Image = tf.keras.layers.Add()([Main_Image, Shortcut_Image])

    if dropout_prob != 0:
        Output_Image = tf.keras.layers.Dropout(rate = dropout_prob)(Output_Image)

    Output_Image = tf.nn.relu(Output_Image)
    
    return Output_Image

In [8]:
def Convolutional_Block(Input_Image, n_filters, dropout_prob = 0, pooling = False):
    

    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()

    # Shortcut path
    Shortcut_Image = Input_Image
    Shortcut_Image = tf.keras.layers.Conv2D(n_filters, (3,3), strides = (2,2), kernel_initializer= W_init)(Shortcut_Image) # stride leads to a reduction in size
    Shortcut_Image = tf.keras.layers.BatchNormalization(axis = 1)(Shortcut_Image)

    # Main path
    Main_Image = Input_Image
    
    Main_Image = tf.keras.layers.Conv2D(n_filters, (3,3), strides = (2,2), kernel_initializer= W_init)(Main_Image)
    Main_Image = tf.keras.layers.BatchNormalization(axis = 1)(Main_Image)
    Main_Image = tf.nn.relu(Main_Image)

    Main_Image = tf.keras.layers.Conv2D(n_filters, (3,3), kernel_initializer= W_init, padding = 'same')(Main_Image) # padding is needed to keep the same dimensions
    Main_Image = tf.keras.layers.BatchNormalization(axis = 1)(Main_Image)


    # Output: Relu activation from (Shortcut path + Main path)
    Output_Image = tf.keras.layers.Add()([Main_Image, Shortcut_Image])

    if pooling:
        Output_Image = tf.keras.layers.MaxPool2D((2,2))(Output_Image)

    if dropout_prob != 0:
        Output_Image = tf.keras.layers.Dropout(rate = dropout_prob)(Output_Image)

    Output_Image = tf.nn.relu(Output_Image)
    
    return Output_Image

In [9]:
def ResNet18(init_filters = 64, drop_prob = 0.1, dense_nodes = 1000, ExtraPooling = False):
    '''This function defines the original ResNet18 network'''
    
    # initialization of the weights
    W_init = tf.initializers.GlorotUniform()
    
    # Input
    X_Input = tf.keras.layers.Input(shape = (256,256, 3))
    
    # Convolutional layer 1
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_Input)
    X = tf.keras.layers.Conv2D(init_filters, (7,7), strides=(2, 2), kernel_initializer= W_init)(X)
    X = tf.keras.layers.BatchNormalization(axis = 1)(X)
    X = tf.nn.relu(X)
    
    # Convolutional layer 2
    X = tf.keras.layers.ZeroPadding2D((1, 1))(X)
    X = tf.keras.layers.MaxPool2D((3,3), strides = (2,2))(X)
    X = Identity_Block(X, init_filters, dropout_prob = drop_prob)
    X = Identity_Block(X, init_filters, dropout_prob = 0)

    # Convolutional layer 3
    X = tf.keras.layers.ZeroPadding2D((1, 1))(X)
    X = Convolutional_Block(X, 2* init_filters, dropout_prob = drop_prob, pooling = ExtraPooling)
    X = Identity_Block(X, 2* init_filters, dropout_prob = 0)

    # Convolutional layer 4
    X = tf.keras.layers.ZeroPadding2D((1, 1))(X)
    X = Convolutional_Block(X,  4* init_filters, dropout_prob = drop_prob, pooling = ExtraPooling)
    X = Identity_Block(X, 4* init_filters, dropout_prob = 0)

    # Convolutional layer 5
    X = tf.keras.layers.ZeroPadding2D((1, 1))(X)
    X = Convolutional_Block(X, 8* init_filters, dropout_prob = drop_prob)
    X = Identity_Block(X, 8* init_filters, dropout_prob = 0)

    # Output layer
    X = tf.keras.layers.AveragePooling2D((2,2))(X)
    X = tf.keras.layers.Flatten()(X)
    X = tf.keras.layers.Dense(dense_nodes, activation= tf.nn.relu)(X)
    X_Output = tf.keras.layers.Dense(2, activation= tf.nn.softmax)(X) # dense layer where the chance on benign and malignant is represented

    # define the model
    model = tf.keras.Model(inputs = X_Input, outputs = X_Output)
      
    return model

In [10]:
# loss function
def Bin_CrossEntropy_Loss(pred_labels, true_labels, GlobalBatchSize):

    true_labels_pos = true_labels[:,0]
    pred_labels_pos = pred_labels[:,0]

    loss_object = tf.keras.losses.BinaryCrossentropy(reduction= tf.keras.losses.Reduction.NONE)
    loss = loss_object([true_labels_pos], [pred_labels_pos])[0]
    return (loss/ GlobalBatchSize)

In [11]:
def train_network(TrainImages, TrainLabels, TestImages, TestLabels, 
                  Drop_Prob = 0.1, Init_Filters = 64, Dense_Nodes = 1000, extra_pooling = False, batch_size = 3, loss_function = 'BinCrossEntr', optim = 'Adam', 
                  learning_rate = tf.Variable(1e-5, dtype=tf.float32), MAX_EPOCH = 10, SaveResults = True):
    '''
    This function trains the UNet on the indicated train data with corresponding annotations
    At the end the trained model is being saved
    '''

    # define the train and test batches that can be fed into the network
    # global batch size defines the batch size over all availabel GPU's
    print('Creating distributed data')
    Global_batch_size = batch_size * strategy.num_replicas_in_sync
    train_batch_data  = tf.data.Dataset.from_tensor_slices((TrainImages, TrainLabels)).shuffle(TrainImages.shape[0]).batch(Global_batch_size) 
    test_batch_data = tf.data.Dataset.from_tensor_slices((TestImages, TestLabels)).batch(Global_batch_size) 

    # distribute the data over the different GPU's
    train_dist_data =  strategy.experimental_distribute_dataset(train_batch_data)
    test_dist_data =  strategy.experimental_distribute_dataset(test_batch_data)

    # define the model that will be used for training and for testing
    # the model, optimisation and loss have to be distributed among GPU's
    tf.compat.v1.reset_default_graph()
    with strategy.scope():

        # model
        print('Defining the model')
        model = ResNet18(init_filters = Init_Filters, drop_prob = Drop_Prob, dense_nodes= Dense_Nodes, ExtraPooling= extra_pooling)
        
        # loss
        print('Defining loss')
        def compute_loss(PredictedLabels, TrueLabels):
            if loss_function == 'BinCrossEntr':
                loss = Bin_CrossEntropy_Loss(PredictedLabels, TrueLabels, Global_batch_size)
            return loss

        # optimization
        steps_per_epoch = int(TrainImages.shape[0]/Global_batch_size)
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate= learning_rate, decay_steps= MAX_EPOCH*steps_per_epoch*0.25, decay_rate=0.2, staircase = True)
        print('Defining optimization')
        if optim == 'Adam':
            train_op = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
        elif optim == 'sgd':
            train_op = tf.keras.optimizers.SGD(learning_rate = lr_schedule)

        # defining the metrics
        print('Defining the metrics')
        test_roc_auc = tf.keras.metrics.AUC(curve = 'ROC')
        test_pr_auc = tf.keras.metrics.AUC(curve = 'PR')

    # one train step is a step in which one batch of data is fed to every GPU
    def Train_Step(input):

        with tf.GradientTape() as tape:
            train_images_batch, train_labels_batch = input
            # make prediction with model
            pred_train_labels = model(train_images_batch, training = True)
            # compute loss
            train_err = compute_loss(pred_train_labels, train_labels_batch)
            
        # update model
        train_weights = model.trainable_variables
        gradients = tape.gradient(train_err, train_weights)
        train_op.apply_gradients(zip(gradients, train_weights))

    # for the last epoch some testing has to be done, in a test step one batch of test data is fed to every GPU
    def Test_Step(input):
        test_images_batch, test_labels_batch = input

        # make prediction with model
        pred_test_labels = model(test_images_batch, training = False)
        # compute loss
        test_err = compute_loss(pred_test_labels, test_labels_batch)
        
        # compute auc and aupr score
        temp_test_labels = test_labels_batch[:,0]
        temp_test_pred_labels = pred_test_labels[:,0]
        test_roc_auc.update_state(temp_test_labels, temp_test_pred_labels)
        test_pr_auc.update_state(temp_test_labels, temp_test_pred_labels)

        # the error per replica is returned
        return test_err, test_roc_auc.result(), test_pr_auc.result()

    @tf.function
    def distributed_train_step(dataset_inputs):
        strategy.run(Train_Step, args=(dataset_inputs,))


    @tf.function
    def distributed_test_step(dataset_inputs):
        per_replica_losses, per_replica_roc_auc, per_replica_pr_auc = strategy.run(Test_Step, args=(dataset_inputs,))
        return per_replica_losses, per_replica_roc_auc, per_replica_pr_auc


    # the train and test steps now have to be performed with the distributed strategy
    print('Training')
    for epo in range(1,MAX_EPOCH+1):
        
        # go over all global batches
        for train_input_data in train_dist_data:
            distributed_train_step(train_input_data)

        # some testing has to be done at the last epoch
        if epo == MAX_EPOCH:
            print('Testing')
            n_test_steps = 0
            total_test_loss = 0
            total_test_auc = 0
            total_test_aupr = 0
            for test_input_data in test_dist_data:
                n_test_steps+=1
                per_replica_test_losses, per_replica_test_roc_auc, per_replica_test_pr_auc = distributed_test_step(test_input_data)
                total_test_loss += strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_test_losses, axis=None)
                total_test_auc += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_roc_auc, axis=None)
                total_test_aupr += strategy.reduce(tf.distribute.ReduceOp.MEAN, per_replica_test_pr_auc, axis=None)

            total_test_loss = total_test_loss/n_test_steps
            total_test_auc = total_test_auc/n_test_steps
            total_test_aupr = total_test_aupr/n_test_steps   

    return total_test_loss, total_test_auc, total_test_aupr   

The functions beneith can be used to perform a gridsearch over different combinations of parameters to define the parametercombination that gives the best results in terms of performance and training time.

In [12]:
# gridsearch: look for the most optimal hyperparameters, making use of cross-validation

# for this a separate validation set has to be defined
# three fold cross-validation is performed in this case
def ThreeFoldSplit(n_split):
    '''
    This function calculates a three-fold split of the train_images and train_annotations
    Depending on n_split another train and validation set is defined
    '''
    
    a = train_images.shape[0]
    b = int(a/3)
    
    val_images = train_images[n_split*b:(n_split+1)*b]
    val_labels = train_labels[n_split*b:(n_split+1)*b]
    
    # split the train set in to three parts
    if n_split == 0:
        tr_images = train_images[b:]
        tr_labels = train_labels[b:]
        
    if n_split == 1:
        tr_images = np.vstack((train_images[0:b], train_images[2*b:]))
        tr_labels = np.vstack((train_labels[0:b], train_labels[2*b:]))
        
    if n_split == 2:
        tr_images = train_images[0:2*b]
        tr_labels = train_labels[0:2*b]
    
    return tr_images, tr_labels, val_images, val_labels

In [16]:
# function to choose the best hyperparameter combination based on model performance in terms of the ROC AUC score
def Hyperparam_Optimization(lr_list, Batch_Size_list, dropout_list, Dense_Nodes_list, MaxPool_list, optim_list, 
                            init_filters_list, epochs_list, savefile):

    n = 0
    file = open(savefile,'w') 
    start = time.time()
    for LearnRate in lr_list:
        for BatchSize in Batch_Size_list:
            for Dropout in dropout_list:
                for DenseNodes in Dense_Nodes_list:
                    for MaxPool in MaxPool_list:
                        for Optim in optim_list:
                            for InitFilters in init_filters_list:
                                for Epoch in epochs_list:
                                
                                    n +=1
                                    print('Evaluating parameter combination {} ...'.format(n))
                                        
                                    AUC_list = []
                                    AUPR_list = []
                                    Loss_list = []
                                    time_list = []
                                    # 3-fold cross-validation is used to evaluate the model 
                                    # for a certain param combination 
                                    for fold in range(3):
                                            
                                        tr_images, tr_labels, val_images, val_labels = ThreeFoldSplit(fold)
                                        
                                        # fitting the model to the data for a certain fold and defining the auc, aupr and loss
                                        # these values can later on be compared for different param combinations
                                        start_time = time.time()
                                        
                                        val_loss, val_auc, val_aupr = train_network(tr_images, tr_labels, val_images, val_labels, Drop_Prob = Dropout, Init_Filters = InitFilters, 
                                                                                    Dense_Nodes = DenseNodes, extra_pooling = MaxPool, batch_size = BatchSize, optim = Optim, 
                                                                                    learning_rate = tf.Variable(LearnRate, dtype=tf.float32), MAX_EPOCH = Epoch, SaveResults = False)
                                        
                                        end_time = time.time()
                                            
                                        # creat a 1D numpy array with the predicted and true outputs   
                                        AUC_list.append(val_auc)
                                        AUPR_list.append(val_aupr)
                                        Loss_list.append(val_loss) 
                                        time_list.append(end_time-start_time) 
                                        
                                    # calculate the mean and standard-dev of the AUC, AUPR, loss and time for all three folds
                                    mean_AUC = np.mean(np.array(AUC_list), axis = 0)
                                    std_AUC = np.std(np.array(AUC_list), axis = 0)
                                    mean_AUPR = np.mean(np.array(AUPR_list), axis = 0)
                                    std_AUPR = np.std(np.array(AUPR_list), axis = 0)
                                    mean_Loss = np.mean(np.array(Loss_list), axis = 0)
                                    std_Loss = np.std(np.array(Loss_list), axis = 0)
                                    mean_time = np.mean(time_list, axis = 0)
                                        
                                    # print out these values together with the training time for this parameter set
                                    print('Hyperparameter combination:')
                                    print('learn rate {}, batch size {}, dropout {}, dense nodes {}, maxpooling {}, optimization {}, initial filters {} and epochs {}'.format(LearnRate, BatchSize, Dropout, DenseNodes, MaxPool, Optim, InitFilters, Epoch))
                                    print('Results:')
                                    print('AUC: mean: {}, standard deviation: {}'.format(mean_AUC, std_AUC))
                                    print('AUPR: mean: {}, standard deviation: {}'.format(mean_AUPR, std_AUPR)) 
                                    print('Loss: mean: {}, standard deviation: {}'.format(mean_Loss, std_Loss)) 
                                    print('training time per epoch: mean: {}'.format(mean_time))

                                    # save all values in a txt file
                                    file.write('Hyperparameter combination: \n')
                                    file.write('learn rate {}, batch size {}, dropout {}, dense nodes {}, maxpooling {}, optimization {}, initial filters {} and epochs {} \n'.format(LearnRate, BatchSize, Dropout, DenseNodes, MaxPool, Optim, InitFilters, Epoch))
                                    file.write('AUC: mean: {}, standard deviation: {} \n'.format(mean_AUC, std_AUC))
                                    file.write('AUPR: mean: {}, standard deviation: {} \n'.format(mean_AUPR, std_AUPR))
                                    file.write('Loss: mean: {}, standard deviation: {} \n'.format(mean_Loss, std_Loss))
                                    file.write('training time per epoch: mean: {} \n'.format(mean_time))
                                    file.write(' \n')
    print('The gridsearch took {}s'.format(time.time()- start))
    file.close()                                                           

In [14]:
# # different options that have to be tried
# # learn rat
# LR_list = [1e-5, 1e-6]
# # batch size
# BS_list = [3, 6]
# # drop probab
# DP_list = [0.3, 0.5]
# # amount of dense nodes at the end
# DN_list = [1000, 100]
# # extra maxpooling or not
# EMP_list = [True, False]
# # optimisation function
# OF_list = ['Adam', 'sgd']
# # initial amount of filters
# IF_list = [64, 32]
# # amount of epochs
# EP_list = [10,30]
# # alpha and gamma, parameters of the focal loss function

# Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
#                         Dense_Nodes_list = DN_list, MaxPool_list = EMP_list, optim_list = OF_list, init_filters_list = IF_list, 
#                         epochs_list = EP_list, savefile = dir_gridsearch)

In [20]:
# different options that have to be tried
# learn rat
LR_list = [1e-3, 1e-5]
# batch size
BS_list = [6,12,24]
# drop probab
DP_list = [0.3, 0.5]
# amount of dense nodes at the end
DN_list = [1000, 100]
# extra maxpooling or not
EMP_list = [True, False]
# optimisation function
OF_list = ['Adam']
# initial amount of filters
IF_list = [64, 32]
# amount of epochs
EP_list = [1]
# alpha and gamma, parameters of the focal loss function

Hyperparam_Optimization(lr_list = LR_list, Batch_Size_list = BS_list, dropout_list = DP_list, 
                        Dense_Nodes_list = DN_list, MaxPool_list = EMP_list, optim_list = OF_list, init_filters_list = IF_list, 
                        epochs_list = EP_list, savefile = dir_gridsearch)

Evaluating parameter combination 1 ...
Hyperparameter combination:
learn rate 0.001, batch size 3, dropout 0.3, dense nodes 1000, maxpooling True, optimization Adam, initial filters 64 and epochs 1
Creating distributed data
Defining the model
Defining loss
Defining optimization
Defining the metrics
Training
Testing
Creating distributed data
Defining the model
Defining loss
Defining optimization
Defining the metrics
Training
Testing
Creating distributed data
Defining the model
Defining loss
Defining optimization
Defining the metrics
Training
Testing
Hyperparameter combination:
learn rate 0.001, batch size 3, dropout 0.3, dense nodes 1000, maxpooling True, optimization Adam, initial filters 64 and epochs 1
Results:
AUC: mean: 0.4702993929386139, standard deviation: 0.04200298711657524
AUPR: mean: 0.4548572301864624, standard deviation: 0.029058728367090225
Loss: mean: 1.8524786233901978, standard deviation: 1.1383150815963745
training time per epoch: mean: 15.183771133422852
Evaluating p

KeyboardInterrupt: ignored